In [5]:
# Do all imports and installs here
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf,col
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import udf,to_timestamp,hour,dayofmonth,weekofyear,month,year,dayofweek,date_format,row_number
from pyspark.sql import Window
import datetime
from datetime import timedelta
from pyspark.sql.functions import broadcast

In [6]:
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11").\
config("spark.sql.broadcastTimeout", "36000")\
.enableHiveSupport().getOrCreate()

In [7]:
# Read in the data here
#df_spark =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')
#df_spark =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016.sas7bdat')
#df_spark = spark.read.format('com.github.saurfang.sas.spark').load('/data/18-83510-I94-Data-2016/*.sas7bdat')
#df_spark =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/')
#df_spark = spark.read.format('com.github.saurfang.sas.spark').load('/data/*/*/*')

In [4]:
#write to parquet
#df_spark.write.parquet("sas_data")
df_spark=spark.read.parquet("sas_data")
df_spark.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = 

In [5]:
def valuetocategory(value):
    if value == 'ALC': return 'ALCAN, AK'
    elif value == 'ANC': return 'ANCHORAGE, AK'
    elif value == 'BAR': return 'BAKER AAF - BAKER ISLAND, AK'
    elif value=='DAC': return'DALTONS CACHE, AK     '
    elif value=='PIZ': return'DEW STATION PT LAY DEW, AK'
    elif value=='DTH': return'DUTCH HARBOR, AK      '
    elif value=='EGL': return'EAGLE, AK             '
    elif value=='FRB': return'FAIRBANKS, AK         '
    elif value=='HOM': return'HOMER, AK             '           
    elif value=='HYD': return'HYDER, AK             '
    elif value=='JUN': return'JUNEAU, AK            '
    elif value=='5KE': return'KETCHIKAN, AK'
    elif value=='KET': return'KETCHIKAN, AK         '
    elif value=='MOS'	: return	'MOSES POINT INTERMEDIATE, AK'
    elif value=='NIK'	: return	'NIKISKI, AK           '
    elif value=='NOM'	: return	'NOM, AK               '
    elif value=='PKC'	: return	'POKER CREEK, AK       '
    elif value=='ORI'	: return	'PORT LIONS SPB, AK'
    elif value=='SKA'	: return	'SKAGWAY, AK           '
    elif value=='SNP'	: return	'ST. PAUL ISLAND, AK'
    elif value=='TKI'	: return	'TOKEEN, AK'
    elif value=='WRA'	: return	'WRANGELL, AK          '
    elif value=='HSV'	: return	'MADISON COUNTY - HUNTSVILLE, AL'
    elif value=='MOB'	: return	'MOBILE, AL            '
    elif value=='LIA'	: return	'LITTLE ROCK, AR (BPS)'
    elif value=='ROG'	: return	'ROGERS ARPT, AR'
    elif value=='DOU'	: return	'DOUGLAS, AZ           '
    elif value=='LUK'	: return	'LUKEVILLE, AZ         '
    elif value=='MAP'	: return	'MARIPOSA AZ           '
    elif value=='NAC'	: return	'NACO, AZ              '
    elif value=='NOG'	: return	'NOGALES, AZ           '
    elif value=='PHO'	: return	'PHOENIX, AZ           '
    elif value=='POR'	: return	'PORTAL, AZ'
    elif value=='SLU'	: return	'SAN LUIS, AZ          '
    elif value=='SAS'	: return	'SASABE, AZ            '
    elif value=='TUC'	: return	'TUCSON, AZ            '
    elif value=='YUI'	: return	'YUMA, AZ              ' 
    elif value=='AND'	: return	'ANDRADE, CA           '
    elif value=='BUR'	: return	'BURBANK, CA'
    elif value=='CAL'	: return	'CALEXICO, CA          '
    elif value=='CAO'	: return	'CAMPO, CA             ' 
    elif value=='FRE'	: return	'FRESNO, CA            '
    elif value=='ICP'	: return	'IMPERIAL COUNTY, CA   '
    elif value=='LNB'	: return	'LONG BEACH, CA         '
    elif value=='LOS'	: return	'LOS ANGELES, CA       '
    elif value=='BFL'	: return	'MEADOWS FIELD - BAKERSFIELD, CA'
    elif value=='OAK'	: return	'OAKLAND, CA ' 
    elif value=='ONT'	: return	'ONTARIO, CA'
    elif value=='OTM'	: return	'OTAY MESA, CA          '
    elif value=='BLT'	: return	'PACIFIC, HWY. STATION, CA '
    elif value=='PSP'	: return	'PALM SPRINGS, CA'
    elif value=='SAC'	: return	'SACRAMENTO, CA        '
    elif value=='SLS'	: return	'SALINAS, CA (BPS)'
    elif value=='SDP'	: return	'SAN DIEGO, CA'
    elif value=='SFR'	: return	'SAN FRANCISCO, CA     '
    elif value=='SNJ'	: return	'SAN JOSE, CA          '
    elif value=='SLO'	: return	'SAN LUIS OBISPO, CA   '
    elif value=='SLI'	: return	'SAN LUIS OBISPO, CA (BPS)'
    elif value=='SPC'	: return	'SAN PEDRO, CA         '
    elif value=='SYS'	: return	'SAN YSIDRO, CA        '
    elif value=='SAA'	: return	'SANTA ANA, CA         '
    elif value=='STO'	: return	'STOCKTON, CA (BPS)'
    elif value=='TEC'	: return	'TECATE, CA            '
    elif value=='TRV'	: return	'TRAVIS-AFB, CA        '
    elif value=='APA'	: return	'ARAPAHOE COUNTY, CO'
    elif value=='ASE'	: return	'ASPEN, CO #ARPT'
    elif value=='COS'	: return	'COLORADO SPRINGS, CO'
    elif value=='DEN'	: return	'DENVER, CO            '
    elif value=='DRO'	: return	'LA PLATA - DURANGO, CO'
    elif value=='BDL'	: return	'BRADLEY INTERNATIONAL, CT'
    elif value=='BGC'	: return	'BRIDGEPORT, CT        '
    elif value=='GRT'	: return	'GROTON, CT            '
    elif value=='HAR'	: return	'HARTFORD, CT          '
    elif value=='NWH'	: return	'NEW HAVEN, CT         '
    elif value=='NWL'	: return	'NEW LONDON, CT        '
    elif value=='TST'	: return	'NEWINGTON DATA CENTER TEST, CT'
    elif value=='WAS'	: return	'WASHINGTON DC         '
    elif value=='DOV'	: return	'DOVER AFB, DE'
    elif value=='DVD'	: return	'DOVER-AFB, DE         '
    elif value=='WLL'	: return	'WILMINGTON, DE        '
    elif value=='BOC'	: return	'BOCAGRANDE, FL        '
    elif value=='SRQ'	: return	'BRADENTON - SARASOTA, FL'
    elif value=='CAN'	: return	'CAPE CANAVERAL, FL    '
    elif value=='DAB'	: return	'DAYTONA BEACH INTERNATIONAL, FL'
    elif value=='FRN'	: return	'FERNANDINA, FL        '
    elif value=='FTL'	: return	'FORT LAUDERDALE, FL   '
    elif value=='FMY'	: return	'FORT MYERS, FL        '
    elif value=='FPF'	: return	'FORT PIERCE, FL       '
    elif value=='HUR'	: return	'HURLBURT FIELD, FL'
    elif value=='GNV'	: return	'J R ALISON MUNI - GAINESVILLE, FL'
    elif value=='JAC'	: return	'JACKSONVILLE, FL      '
    elif value=='KEY'	: return	'KEY WEST, FL          '
    elif value=='LEE'	: return	'LEESBURG MUNICIPAL AIRPORT, FL'
    elif value=='MLB'	: return	'MELBOURNE, FL'
    elif value=='MIA'	: return	'MIAMI, FL             '
    elif value=='APF'	: return	'NAPLES, FL #ARPT'
    elif value=='OPF'	: return	'OPA LOCKA, FL'
    elif value=='ORL'	: return	'ORLANDO, FL           '
    elif value=='PAN'	: return	'PANAMA CITY, FL       '
    elif value=='PEN'	: return	'PENSACOLA, FL         '
    elif value=='PCF'	: return	'PORT CANAVERAL, FL    '
    elif value=='PEV'	: return	'PORT EVERGLADES, FL   '
    elif value=='PSJ'	: return	'PORT ST JOE, FL       '
    elif value=='SFB'	: return	'SANFORD, FL           '
    elif value=='SGJ'	: return	'ST AUGUSTINE ARPT, FL'
    elif value=='SAU'	: return	'ST AUGUSTINE, FL      '
    elif value=='FPR'	: return	'ST LUCIE COUNTY, FL'
    elif value=='SPE'	: return	'ST PETERSBURG, FL     '
    elif value=='TAM'	: return	'TAMPA, FL             '
    elif value=='WPB'	: return	'WEST PALM BEACH, FL   '
    elif value=='ATL'	: return	'ATLANTA, GA           '
    elif value=='BRU'	: return	'BRUNSWICK, GA         '
    elif value=='AGS'	: return	'BUSH FIELD - AUGUSTA, GA'
    elif value=='SAV'	: return	'SAVANNAH, GA          '
    elif value=='AGA'	: return	'AGANA, GU             '
    elif value=='HHW'	: return	'HONOLULU, HI          '
    elif value=='OGG'	: return	'KAHULUI - MAUI, HI'
    elif value=='KOA'	: return	'KEAHOLE-KONA, HI      '
    elif value=='LIH'	: return	'LIHUE, HI             '
    elif value== 'CID'	: return	'CEDAR RAPIDS/IOWA CITY, IA'
    elif value=='BOI'	: return	'AIR TERM. (GOWEN FLD) BOISE, ID'
    elif value=='EPI'	: return	'EASTPORT, ID          '
    elif value=='IDA'	: return	'FANNING FIELD - IDAHO FALLS, ID'
    elif value=='PTL'	: return	'PORTHILL, ID          '
    elif value=='SPI'	: return	'CAPITAL - SPRINGFIELD, IL'
    elif value=='CHI'	: return	'CHICAGO, IL           '
    elif value=='PIA'	: return	'GREATER PEORIA, IL'
    elif value=='RFD'	: return	'GREATER ROCKFORD, IL'
    elif value=='UGN'	: return	'MEMORIAL - WAUKEGAN, IL'
    elif value=='GAR'	: return	'GARY, IN              '
    elif value=='HMM'	: return	'HAMMOND, IN           '
    elif value=='INP'	: return	'INDIANAPOLIS, IN      '
    elif value=='MRL'	: return	'MERRILLVILLE, IN      '
    elif value=='SBN'	: return	'SOUTH BEND, IN'
    elif value=='ICT'	: return	'MID-CONTINENT - WITCHITA, KS'
    elif value=='LEX'	: return	'BLUE GRASS - LEXINGTON, KY'
    elif value=='LOU'	: return	'LOUISVILLE, KY        '
    elif value=='BTN'	: return	'BATON ROUGE, LA       '
    elif value=='LKC'	: return	'LAKE CHARLES, LA      '
    elif value=='LAK'	: return	'LAKE CHARLES, LA (BPS)'
    elif value=='MLU'	: return	'MONROE, LA'
    elif value=='MGC'	: return	'MORGAN CITY, LA       '
    elif value=='NOL'	: return	'NEW ORLEANS, LA       '
    elif value=='BOS'	: return	'BOSTON, MA            '
    elif value=='GLO'	: return	'GLOUCESTER, MA        '
    elif value=='BED'	: return	'HANSCOM FIELD - BEDFORD, MA'
    elif value=='LYN'	: return	'LYNDEN, WA            '
    elif value=='ADW'	: return	'ANDREWS AFB, MD'
    elif value=='BAL'	: return	'BALTIMORE, MD         '
    elif value=='MKG'	: return	'MUSKEGON, MD'
    elif value=='PAX'	: return	'PATUXENT RIVER, MD    '
    elif value=='BGM'	: return	'BANGOR, ME            '
    elif value=='BOO'	: return	'BOOTHBAY HARBOR, ME   '
    elif value=='BWM'	: return	'BRIDGEWATER, ME       '
    elif value=='BCK'	: return	'BUCKPORT, ME          '
    elif value=='CLS'	: return	'CALAIS, ME   '
    elif value=='CRB'	: return	'CARIBOU, ME           '
    elif value=='COB'	: return	'COBURN GORE, ME       '
    elif value=='EST'	: return	'EASTCOURT, ME         '
    elif value=='EPT'	: return	'EASTPORT MUNICIPAL, ME'
    elif value=='EPM'	: return	'EASTPORT, ME          '
    elif value=='FOR'	: return	'FOREST CITY, ME       '
    elif value=='FTF'	: return	'FORT FAIRFIELD, ME    '
    elif value=='FTK'	: return	'FORT KENT, ME         '
    elif value=='HML'	: return	'HAMIIN, ME            '
    elif value=='HTM'	: return	'HOULTON, ME           '
    elif value=='JKM'	: return	'JACKMAN, ME           '
    elif value=='KAL'	: return	'KALISPEL, MT          '
    elif value=='LIM'	: return	'LIMESTONE, ME         '
    elif value=='LUB'	: return	'LUBEC, ME             '
    elif value=='MAD'	: return	'MADAWASKA, ME         '
    elif value=='POM'	: return	'PORTLAND, ME          '
    elif value=='RGM'	: return	'RANGELEY, ME (BPS)'
    elif value=='SBR'	: return	'SOUTH BREWER, ME      '
    elif value=='SRL'	: return	'ST AURELIE, ME        '
    elif value=='SPA'	: return	'ST PAMPILE, ME        '
    elif value=='VNB'	: return	'VAN BUREN, ME         '
    elif value=='VCB'	: return	'VANCEBORO, ME         '
    elif value=='AGN'	: return	'ALGONAC, MI           '
    elif value== 'ALP'	: return	'ALPENA, MI            '
    elif value=='BCY'	: return	'BAY CITY, MI          '
    elif value=='DET'	: return	'DETROIT, MI           '
    elif value=='GRP'	: return	'GRAND RAPIDS, MI'
    elif value=='GRO'	: return	'GROSSE ISLE, MI       '
    elif value=='ISL'	: return	'ISLE ROYALE, MI       '
    elif value=='MRC'	: return	'MARINE CITY, MI       '
    elif value=='MRY'	: return	'MARYSVILLE, MI        '
    elif value=='PTK'	: return	'OAKLAND COUNTY - PONTIAC, MI'
    elif value=='PHU'	: return	'PORT HURON, MI        '
    elif value=='RBT'	: return	'ROBERTS LANDING, MI   '
    elif value=='SAG'	: return	'SAGINAW, MI           '
    elif value=='SSM'	: return	'SAULT STE. MARIE, MI  '
    elif value=='SCL'	: return	'ST CLAIR, MI          '
    elif value=='YIP'	: return	'WILLOW RUN - YPSILANTI, MI'
    elif value=='BAU'	: return	'BAUDETTE, MN          '
    elif value=='CAR'	: return	'CARIBOU MUNICIPAL AIRPORT, MN'
    elif value=='GTF'	: return	'Collapsed into INT, MN'
    elif value=='INL'	: return	'Collapsed into INT, MN'
    elif value=='CRA'	: return	'CRANE LAKE, MN        '
    elif value=='MIC'	: return	'CRYSTAL MUNICIPAL AIRPORT, MN'
    elif value== 'DUL'	: return	'DULUTH, MN            '
    elif value=='ELY'	: return	'ELY, MN               '
    elif value=='GPM'	: return	'GRAND PORTAGE, MN     '
    elif value=='SVC'	: return	'GRANT COUNTY - SILVER CITY, MN'
    elif value=='INT'	: return	'INT''L FALLS, MN      '
    elif value=='LAN'	: return	'LANCASTER, MN         '
    elif value=='MSP'	: return	'MINN./ST PAUL, MN     '
    elif value=='LIN'	: return	'NORTHERN SVC CENTER, MN   '
    elif value=='NOY'	: return	'NOYES, MN             '
    elif value=='PIN'	: return	'PINE CREEK, MN        '
    elif value=='48Y'	: return	'PINECREEK BORDER ARPT, MN'
    elif value=='RAN'	: return	'RAINER, MN            '
    elif value=='RST'	: return	'ROCHESTER, MN'
    elif value=='ROS'	: return	'ROSEAU, MN            '
    elif value=='SPM'	: return	'ST PAUL, MN           '
    elif value=='WSB'	: return	'WARROAD INTL, SPB, MN'
    elif value=='WAR'	: return	'WARROAD, MN           '
    elif value=='KAN'	: return	'KANSAS CITY, MO       '
    elif value=='SGF'	: return	'SPRINGFIELD-BRANSON, MO'
    elif value=='STL'	: return	'ST LOUIS, MO          '
    elif value=='WHI'	: return	'WHITETAIL, MT         '
    elif value=='WHM'	: return	'WILD HORSE, MT        '
    elif value=='GPT'	: return	'BILOXI REGIONAL, MS'
    elif value=='GTR'	: return	'GOLDEN TRIANGLE LOWNDES CNTY, MS'
    elif value=='GUL'	: return	'GULFPORT, MS          '
    elif value=='PAS'	: return	'PASCAGOULA, MS        '
    elif value=='JAN'	: return	'THOMPSON FIELD - JACKSON, MS'
    elif value=='BIL'	: return	'BILLINGS, MT          '
    elif value=='BTM'	: return	'BUTTE, MT             '
    elif value=='CHF'	: return	'CHIEF MT, MT          '
    elif value=='CTB'	: return	'CUT BANK MUNICIPAL, MT'
    elif value=='CUT'	: return	'CUT BANK, MT          '
    elif value=='DLB'	: return	'DEL BONITA, MT        '
    elif value=='EUR'	: return	'EUREKA, MT (BPS)'
    elif value=='BZN'	: return	'GALLATIN FIELD - BOZEMAN, MT'
    elif value=='FCA'	: return	'GLACIER NATIONAL PARK, MT'
    elif value=='GGW'	: return	'GLASGOW, MT           '
    elif value=='GRE'	: return	'GREAT FALLS, MT       '
    elif value=='HVR'	: return	'HAVRE, MT             '
    elif value=='HEL'	: return	'HELENA, MT            '
    elif value=='LWT'	: return	'LEWISTON, MT          '
    elif value=='MGM'	: return	'MORGAN, MT            '
    elif value=='OPH'	: return	'OPHEIM, MT            '
    elif value=='PIE'	: return	'PIEGAN, MT            '
    elif value=='RAY'	: return	'RAYMOND, MT           '
    elif value=='ROO'	: return	'ROOSVILLE, MT         '
    elif value=='SCO'	: return	'SCOBEY, MT            '
    elif value=='SWE'	: return	'SWEETGTASS, MT        '
    elif value=='TRL'	: return	'TRIAL CREEK, MT       '
    elif value=='TUR'	: return	'TURNER, MT            '
    elif value=='WCM'	: return	'WILLOW CREEK, MT      '
    elif value=='CLT'	: return	'CHARLOTTE, NC         '
    elif value=='FAY'	: return	'FAYETTEVILLE, NC'
    elif value=='MRH'	: return	'MOREHEAD CITY, NC     '
    elif value=='FOP'	: return	'MORRIS FIELDS AAF, NC'
    elif value=='GSO'	: return	'PIEDMONT TRIAD INTL AIRPORT, NC'
    elif value=='RDU'	: return	'RALEIGH/DURHAM, NC    '
    elif value=='SSC'	: return	'SHAW AFB - SUMTER, NC'
    elif value=='WIL'	: return	'WILMINGTON, NC        '
    elif value=='AMB'	: return	'AMBROSE, ND           '
    elif value=='ANT'	: return	'ANTLER, ND            '
    elif value=='CRY'	: return	'CARBURY, ND           '
    elif value=='DNS'	: return	'DUNSEITH, ND          '
    elif value=='FAR'	: return	'FARGO, ND             '
    elif value=='FRT'	: return	'FORTUNA, ND           '
    elif value=='GRF'	: return	'GRAND FORKS, ND       '
    elif value=='HNN'	: return	'HANNAH, ND            '
    elif value=='HNS'	: return	'HANSBORO, ND          '
    elif value=='MAI'	: return	'MAIDA, ND             '
    elif value=='MND'	: return	'MINOT, ND             '
    elif value=='NEC'	: return	'NECHE, ND             '
    elif value=='NOO'	: return	'NOONAN, ND            '
    elif value=='NRG'	: return	'NORTHGATE, ND         '
    elif value=='PEM'	: return	'PEMBINA, ND           '
    elif value=='SAR'	: return	'SARLES, ND            '
    elif value=='SHR'	: return	'SHERWOOD, ND          '
    elif value=='SJO'	: return	'ST JOHN, ND           '
    elif value=='WAL'	: return	'WALHALLA, ND          '
    elif value=='WHO'	: return	'WESTHOPE, ND          '
    elif value=='WND'	: return	'WILLISTON, ND         '
    elif value=='OMA'	: return	'OMAHA, NE             '
    elif value=='LEB'	: return	'LEBANON, NH           '
    elif value=='MHT'	: return	'MANCHESTER, NH'
    elif value=='PNH'	: return	'PITTSBURG, NH         '
    elif value=='PSM'	: return	'PORTSMOUTH, NH        '
    elif value=='BYO'	: return	'BAYONNE, NJ           '
    elif value=='CNJ'	: return	'CAMDEN, NJ            '
    elif value=='HOB'	: return	'HOBOKEN, NJ           '
    elif value=='JER'	: return	'JERSEY CITY, NJ       '
    elif value=='WRI'	: return	'MC GUIRE AFB - WRIGHTSOWN, NJ'
    elif value=='MMU'	: return	'MORRISTOWN, NJ'
    elif value=='NEW'	: return	'NEWARK/TETERBORO, NJ  '
    elif value=='PER'	: return	'PERTH AMBOY, NJ       '
    elif value=='ACY'	: return	'POMONA FIELD - ATLANTIC CITY, NJ'
    elif value=='ALA'	: return	'ALAMAGORDO, NM (BPS)'
    elif value=='ABQ'	: return	'ALBUQUERQUE, NM       '
    elif value=='ANP'	: return	'ANTELOPE WELLS, NM    '
    elif value=='CRL'	: return	'CARLSBAD, NM          '
    elif value=='COL'	: return	'COLUMBUS, NM          '
    elif value=='CDD'	: return	'CRANE LAKE - ST. LOUIS CNTY, NM'
    elif value=='DNM'	: return	'DEMING, NM (BPS)'
    elif value=='LAS'	: return	'LAS CRUCES, NM        '
    elif value=='LOB'	: return	'LORDSBURG, NM (BPS)'
    elif value=='RUI'	: return	'RUIDOSO, NM'
    elif value=='STR'	: return	'SANTA TERESA, NM      '
    elif value=='RNO'	: return	'CANNON INTL - RENO/TAHOE, NV'
    elif value=='FLX'	: return	'FALLON MUNICIPAL AIRPORT, NV'
    elif value=='LVG'	: return	'LAS VEGAS, NV         '
    elif value=='REN'	: return	'RENO, NV              '
    elif value=='ALB'	: return	'ALBANY, NY            '
    elif value=='AXB'	: return	'ALEXANDRIA BAY, NY    '
    elif value=='BUF'	: return	'BUFFALO, NY           '
    elif value=='CNH'	: return	'CANNON CORNERS, NY'
    elif value=='CAP'	: return	'CAPE VINCENT, NY      '
    elif value=='CHM'	: return	'CHAMPLAIN, NY         '
    elif value=='CHT'	: return	'CHATEAUGAY, NY        '
    elif value=='CLA'	: return	'CLAYTON, NY           '
    elif value=='FTC'	: return	'FORT COVINGTON, NY    '
    elif value=='LAG'	: return	'LA GUARDIA, NY        '
    elif value=='LEW'	: return	'LEWISTON, NY          '
    elif value=='MAS'	: return	'MASSENA, NY           '
    elif value=='MAG'	: return	'MCGUIRE AFB, NY       '
    elif value=='MOO'	: return	'MOORES, NY            '
    elif value== 'MRR'	: return	'MORRISTOWN, NY        '
    elif value=='NYC'	: return	'NEW YORK, NY          '
    elif value=='NIA'	: return	'NIAGARA FALLS, NY     '
    elif value=='OGD'	: return	'OGDENSBURG, NY        '
    elif value=='OSW'	: return	'OSWEGO, NY            '
    elif value=='ELM'	: return	'REGIONAL ARPT - HORSEHEAD, NY'
    elif value=='ROC'	: return	'ROCHESTER, NY         '
    elif value=='ROU'	: return	'ROUSES POINT, NY      '
    elif value== 'SWF'	: return	'STEWART - ORANGE CNTY, NY'
    elif value=='SYR'	: return	'SYRACUSE, NY          '
    elif value=='THO'	: return	'THOUSAND ISLAND BRIDGE, NY'
    elif value=='TRO'	: return	'TROUT RIVER, NY       '
    elif value=='WAT'	: return	'WATERTOWN, NY         '
    elif value=='HPN'	: return	'WESTCHESTER - WHITE PLAINS, NY'
    elif value=='WRB'	: return	'WHIRLPOOL BRIDGE, NY'
    elif value=='YOU'	: return	'YOUNGSTOWN, NY        '
    elif value=='AKR'	: return	'AKRON, OH             '
    elif value==  'ATB'	: return	'ASHTABULA, OH         '
    elif value== 'CIN'	: return	'CINCINNATI, OH        '
    elif value== 'CLE'	: return	'CLEVELAND, OH         '
    elif value== 'CLM'	: return	'COLUMBUS, OH          '
    elif value== 'LOR'	: return	'LORAIN, OH            '
    elif value=='MBO'	: return	'MARBLE HEADS, OH      '
    elif value== 'SDY'	: return	'SANDUSKY, OH          '
    elif value== 'TOL'	: return	'TOLEDO, OH            '
    elif value== 'OKC'	: return	'OKLAHOMA CITY, OK     '
    elif value== 'TUL'	: return	'TULSA, OK'
    elif value=='AST'	: return	'ASTORIA, OR           '
    elif value=='COO'	: return	'COOS BAY, OR          '
    elif value=='HIO'	: return	'HILLSBORO, OR'
    elif value== 'MED'	: return	'MEDFORD, OR           '
    elif value== 'NPT'	: return	'NEWPORT, OR           '
    elif value== 'POO'	: return	'PORTLAND, OR          '
    elif value=='PUT'	: return	'PUT-IN-BAY, OH        '
    elif value== 'RDM'	: return	'ROBERTS FIELDS - REDMOND, OR'
    elif value=='ERI'	: return	'ERIE, PA              '
    elif value== 'MDT'	: return	'HARRISBURG, PA'
    elif value=='HSB'	: return	'HARRISONBURG, PA      '
    elif value== 'PHI'	: return	'PHILADELPHIA, PA      '
    elif value== 'PIT'	: return	'PITTSBURG, PA         '
    elif value== 'AGU'	: return	'AGUADILLA, PR         '
    elif value== 'BQN'	: return	'BORINQUEN - AGUADILLO, PR'
    elif value== 'JCP'	: return	'CULEBRA - BENJAMIN RIVERA, PR'
    elif value== 'ENS'	: return	'ENSENADA, PR          '
    elif value== 'FAJ'	: return	'FAJARDO, PR           '
    elif value== 'HUM'	: return	'HUMACAO, PR           '
    elif value== 'JOB'	: return	'JOBOS, PR             '
    elif value== 'MAY'	: return	'MAYAGUEZ, PR          '
    elif value=='PON'	: return	'PONCE, PR             '
    elif value== 'PSE'	: return	'PONCE-MERCEDITA, PR'
    elif value=='SAJ'	: return	'SAN JUAN, PR          '
    elif value== 'VQS'	: return	'VIEQUES-ARPT, PR'
    elif value=='PRO'	: return	'PROVIDENCE, RI        '
    elif value=='PVD'	: return	'THEODORE FRANCIS - WARWICK, RI'
    elif value== 'CHL'	: return	'CHARLESTON, SC        '
    elif value== 'CAE'	: return	'COLUMBIA, SC #ARPT'
    elif value== 'GEO'	: return	'GEORGETOWN, SC        '
    elif value== 'GSP'	: return	'GREENVILLE, SC'
    elif value== 'GRR'	: return	'GREER, SC'
    elif value=='MYR'	: return	'MYRTLE BEACH, SC'
    elif value== 'SPF'	: return	'BLACK HILLS, SPEARFISH, SD'
    elif value=='HON'	: return	'HOWES REGIONAL ARPT - HURON, SD'
    elif value=='SAI'	: return	'SAIPAN, SPN           '
    elif value== 'TYS'	: return	'MC GHEE TYSON - ALCOA, TN'
    elif value=='MEM'	: return	'MEMPHIS, TN           '
    elif value==  'NSV'	: return	'NASHVILLE, TN         '
    elif value== 'TRI'	: return	'TRI CITY ARPT, TN'
    elif value=='ADS'	: return	'ADDISON AIRPORT- ADDISON, TX'
    elif value=='ADT'	: return	'AMISTAD DAM, TX       '
    elif value== 'ANZ'	: return	'ANZALDUAS, TX'
    elif value== 'AUS'	: return	'AUSTIN, TX            '
    elif value==  'BEA'	: return	'BEAUMONT, TX          '
    elif value=='BBP'	: return	'BIG BEND PARK, TX (BPS)'
    elif value=='SCC'	: return	'BP SPEC COORD. CTR, TX'
    elif value=='BTC'	: return	'BP TACTICAL UNIT, TX  ' 
    elif value=='BOA'	: return	'BRIDGE OF AMERICAS, TX'
    elif value=='BRO'	: return	'BROWNSVILLE, TX       '
    elif value== 'CRP'	: return	'CORPUS CHRISTI, TX    '
    elif value== 'DAL'	: return	'DALLAS, TX            '
    elif value== 'DLR'	: return	'DEL RIO, TX           '
    elif value=='DNA'	: return	'DONNA, TX'
    elif value=='EGP'	: return	'EAGLE PASS, TX        '
    elif value=='ELP'	: return	'EL PASO, TX           '
    elif value== 'FAB'	: return	'FABENS, TX            '
    elif value== 'FAL'	: return	'FALCON HEIGHTS, TX    '
    elif value=='FTH'	: return	'FORT HANCOCK, TX      '
    elif value=='AFW'	: return	'FORT WORTH ALLIANCE, TX'
    elif value== 'FPT'	: return	'FREEPORT, TX          '
    elif value== 'GAL'	: return	'GALVESTON, TX         '
    elif value== 'HLG'	: return	'HARLINGEN, TX         '
    elif value=='HID'	: return	'HIDALGO, TX           '
    elif value== 'HOU'	: return	'HOUSTON, TX           '
    elif value== 'LLB'	: return	'JUAREZ-LINCOLN BRIDGE, TX'
    elif value== 'LCB'	: return	'LAREDO COLUMBIA BRIDGE, TX'
    elif value=='LRN'	: return	'LAREDO NORTH, TX      '
    elif value== 'LAR'	: return	'LAREDO, TX            '
    elif value== 'LSE'	: return	'LOS EBANOS, TX        '
    elif value== 'IND'	: return	'LOS INDIOS, TX'
    elif value== 'LOI'	: return	'LOS INDIOS, TX        '
    elif value== 'MRS'	: return	'MARFA, TX (BPS)'
    elif value== 'MCA'	: return	'MCALLEN, TX           '
    elif value==  'MAF'	: return	'ODESSA REGIONAL, TX'
    elif value=='PDN'	: return	'PASO DEL NORTE,TX     '
    elif value=='PBB'	: return	'PEACE BRIDGE, NY      '
    elif value== 'PHR'	: return	'PHARR, TX             '
    elif value== 'PAR'	: return	'PORT ARTHUR, TX       '
    elif value== 'ISB'	: return	'PORT ISABEL, TX       '
    elif value== 'POE'	: return	'PORT OF EL PASO, TX   '
    elif value== 'PRE'	: return	'PRESIDIO, TX          '
    elif value== 'PGR'	: return	'PROGRESO, TX          '
    elif value== 'RIO'	: return	'RIO GRANDE CITY, TX   '
    elif value=='ROM'	: return	'ROMA, TX              '
    elif value=='SNA'	: return	'SAN ANTONIO, TX       '
    elif value==  'SNN'	: return	'SANDERSON, TX         '
    elif value== 'VIB'	: return	'VETERAN INTL BRIDGE, TX'
    elif value== 'YSL'	: return	'YSLETA, TX            '
    elif value=='CHA'	: return	'CHARLOTTE AMALIE, VI  '
    elif value=='CHR'	: return	'CHRISTIANSTED, VI     '
    elif value== 'CRU'	: return	'CRUZ BAY, ST JOHN, VI '
    elif value== 'FRK'	: return	'FREDERIKSTED, VI      '
    elif value==  'STT'	: return	'ST THOMAS, VI         '
    elif value== 'LGU'	: return	'CACHE AIRPORT - LOGAN, UT'
    elif value==  'SLC'	: return	'SALT LAKE CITY, UT    '
    elif value==  'CHO'	: return	'ALBEMARLE CHARLOTTESVILLE, VA'
    elif value== 'DAA'	: return	'DAVISON AAF - FAIRFAX CNTY, VA'
    elif value== 'HOP'	: return	'HOPEWELL, VA          '
    elif value=='HEF'	: return	'MANASSAS, VA #ARPT'
    elif value=='NWN'	: return	'NEWPORT, VA           '
    elif value== 'NOR'	: return	'NORFOLK, VA           '
    elif value== 'RCM'	: return	'RICHMOND, VA          '
    elif value== 'ABS'	: return	'ALBURG SPRINGS, VT    '
    elif value== 'ABG'	: return	'ALBURG, VT            '
    elif value== 'BRG'	: return	'BURLINGTON, VT        '
    elif value== 'CNA'	: return	'CANAAN, VT            '
    elif value== 'DER'	: return	'DERBY LINE, VT (I-91) '
    elif value==  'DLV'	: return	'DERBY LINE, VT (RT. 5)'
    elif value== 'ERC'	: return	'EAST RICHFORD, VT     '
    elif value== 'HIG'	: return	'HIGHGATE SPRINGS, VT  '
    elif value=='MOR'	: return	'MORSES LINE, VT       '
    elif value==  'NPV'	: return	'NEWPORT, VT           '
    elif value== 'NRT'	: return	'NORTH TROY, VT        '
    elif value=='NRN'	: return	'NORTON, VT            '
    elif value=='PIV'	: return	'PINNACLE ROAD, VT     '
    elif value=='RIF'	: return	'RICHFORT, VT          '
    elif value== 'STA'	: return	'ST ALBANS, VT         '
    elif value== 'SWB'	: return	'SWANTON, VT (BP - SECTOR HQ)'
    elif value== 'WBE'	: return	'WEST BERKSHIRE, VT    '
    elif value== 'ABE'	: return	'ABERDEEN, WA          '
    elif value== 'ANA'	: return	'ANACORTES, WA         '
    elif value== 'BEL'	: return	'BELLINGHAM, WA        '
    elif value== 'BLI'	: return	'BELLINGHAM, WASHINGTON #INTL'
    elif value== 'BLA'	: return	'BLAINE, WA            '
    elif value== 'BWA'	: return	'BOUNDARY, WA          '
    elif value=='CUR'	: return	'CURLEW, WA (BPS)'
    elif value=='DVL'	: return	'DANVILLE, WA          '
    elif value== 'EVE'	: return	'EVERETT, WA           '
    elif value=='FER'	: return	'FERRY, WA             '
    elif value=='FRI'	: return	'FRIDAY HARBOR, WA     '
    elif value=='FWA'	: return	'FRONTIER, WA          '
    elif value== 'KLM'	: return	'KALAMA, WA            '
    elif value== 'LAU'	: return	'LAURIER, WA           '
    elif value== 'LON'	: return	'LONGVIEW, WA          '
    elif value=='MET'	: return	'METALINE FALLS, WA    '
    elif value=='MWH'	: return	'MOSES LAKE GRANT COUNTY ARPT, WA'
    elif value== 'NEA'	: return	'NEAH BAY, WA          '
    elif value== 'NIG'	: return	'NIGHTHAWK, WA         '
    elif value==  'OLY'	: return	'OLYMPIA, WA           '
    elif value=='ORO'	: return	'OROVILLE, WA          '
    elif value== 'PWB'	: return	'PASCO, WA             '
    elif value== 'PIR'	: return	'POINT ROBERTS, WA     '
    elif value=='PNG'	: return	'PORT ANGELES, WA      '
    elif value== 'PTO'	: return	'PORT TOWNSEND, WA     '
    elif value== 'SEA'	: return	'SEATTLE, WA           '
    elif value== 'SPO'	: return	'SPOKANE, WA           '
    elif value== 'SUM'	: return	'SUMAS, WA             '
    elif value== 'TAC'	: return	'TACOMA, WA            '
    elif value== 'PSC'	: return	'TRI-CITIES - PASCO, WA'
    elif value== 'VAN'	: return	'VANCOUVER, WA         '
    elif value==  'AGM'	: return	'ALGOMA, WI            '
    elif value==  'BAY'	: return	'BAYFIELD, WI          '
    elif value== 'GRB'	: return	'GREEN BAY, WI         '
    elif value== 'MNW'	: return	'MANITOWOC, WI         '
    elif value== 'MIL'	: return	'MILWAUKEE, WI         '
    elif value==  'MSN'	: return	'TRUAX FIELD - DANE COUNTY, WI'
    elif value== 'CHS'	: return	'CHARLESTON, WV        '
    elif value=='CLK'	: return	'CLARKSBURG, WV        '
    elif value== 'BLF'	: return	'MERCER COUNTY, WV'
    elif value==  'CSP'	: return	'CASPER, WY            '
    elif value== 'XXX'	: return	'NOT REPORTED/UNKNOWN  ' 
    elif value==  '888'	: return	'UNIDENTIFED AIR / SEAPORT'
    elif value== 'UNK'	: return	'UNKNOWN POE           '
    elif value==  'CLG'	: return	'CALGARY, CANADA       '
    elif value== 'EDA'	: return	'EDMONTON, CANADA      '
    elif value== 'YHC'	: return	'HAKAI PASS, CANADA'
    elif value== 'HAL'	: return	'Halifax, NS, Canada   '
    elif value== 'MON'	: return	'MONTREAL, CANADA      '
    elif value==  'OTT'	: return	'OTTAWA, CANADA        '
    elif value== 'YXE'	: return	'SASKATOON, CANADA'
    elif value== 'TOR'	: return	'TORONTO, CANADA       '
    elif value== 'VCV'	: return	'VANCOUVER, CANADA     '
    elif value=='VIC'	: return	'VICTORIA, CANADA      '
    elif value=='WIN'	: return	'WINNIPEG, CANADA      '
    elif value==  'AMS'	: return	'AMSTERDAM-SCHIPHOL, NETHERLANDS'
    elif value== 'ARB'	: return	'ARUBA, NETH ANTILLES  '
    elif value=='BAN'	: return	'BANKOK, THAILAND      '
    elif value== 'BEI'	: return	'BEICA #ARPT, ETHIOPIA'
    elif value== 'PEK'	: return	'BEIJING CAPITAL INTL, PRC'
    elif value== 'BDA'	: return	'KINDLEY FIELD, BERMUDA'
    elif value== 'BOG'	: return	'BOGOTA, EL DORADO #ARPT, COLOMBIA'
    elif value=='EZE'	: return	'BUENOS AIRES, MINISTRO PIST, ARGENTINA'
    elif value=='CUN'	: return	'CANCUN, MEXICO'
    elif value== 'CRQ'	: return	'CARAVELAS, BA #ARPT, BRAZIL'
    elif value== 'MVD'	: return	'CARRASCO, URUGUAY'
    elif value== 'DUB'	: return	'DUBLIN, IRELAND       '
    elif value=='FOU'	: return	'FOUGAMOU #ARPT, GABON'
    elif value=='FBA'	: return	'FREEPORT, BAHAMAS      '
    elif value=='MTY'	: return	'GEN M. ESCOBEDO, Monterrey, MX'
    elif value=='HMO'	: return	'GEN PESQUEIRA GARCIA, MX'
    elif value== 'GCM'	: return	'GRAND CAYMAN, CAYMAN ISLAND'
    elif value== 'GDL'	: return	'GUADALAJARA, MIGUEL HIDAL, MX'
    elif value== 'HAM'	: return	'HAMILTON, BERMUDA     '
    elif value== 'ICN'	: return	'INCHON, SEOUL KOREA'
    elif value== 'IWA'	: return	'INVALID - IWAKUNI, JAPAN'
    elif value== 'CND'	: return	'KOGALNICEANU, ROMANIA'
    elif value== 'LAH'	: return	'LABUHA ARPT, INDONESIA'
    elif value=='DUR'	: return	'LOUIS BOTHA, SOUTH AFRICA'
    elif value=='MAL'	: return	'MANGOLE ARPT, INDONESIA'
    elif value== 'MDE'	: return	'MEDELLIN, COLOMBIA'
    elif value=='MEX'	: return	'JUAREZ INTL, MEXICO CITY, MX'
    elif value== 'LHR'	: return	'MIDDLESEX, ENGLAND'
    elif value== 'NBO'	: return	'NAIROBI, KENYA        '
    elif value== 'NAS'	: return	'NASSAU, BAHAMAS       '
    elif value== 'NCA'	: return	'NORTH CAICOS, TURK & CAIMAN'
    elif value== 'PTY'	: return	'OMAR TORRIJOS, PANAMA'
    elif value== 'SPV'	: return	'PAPUA, NEW GUINEA'
    elif value=='UIO'	: return	'QUITO (MARISCAL SUCR), ECUADOR'
    elif value==  'RIT'	: return	'ROME, ITALY           '
    elif value== 'SNO'	: return	'SAKON NAKHON #ARPT, THAILAND'
    elif value=='SLP'	: return	'SAN LUIS POTOSI #ARPT, MEXICO'
    elif value=='SAN'	: return	'SAN SALVADOR, EL SALVADOR'
    elif value=='SRO'	: return	'SANTANA RAMOS #ARPT, COLOMBIA'
    elif value=='GRU'	: return	'GUARULHOS INTL, SAO PAULO, BRAZIL'
    elif value=='SHA'	: return	'SHANNON, IRELAND      '
    elif value== 'HIL'	: return	'SHILLAVO, ETHIOPIA'
    elif value== 'TOK'	: return	'TOROKINA #ARPT, PAPUA, NEW GUINEA'
    elif value=='VER'	: return	'VERACRUZ, MEXICO'
    elif value== 'LGW'	: return	'WEST SUSSEX, ENGLAND  '
    elif value=='ZZZ'	: return	'MEXICO Land (Banco de Mexico) '
    elif value== 'CHN'	: return	'No PORT Code (CHN)'
    elif value== 'CNC'	: return	'CANNON CORNERS, NY'
    elif value=='MAA'	: return	'Abu Dhabi'
    elif value=='AG0'	: return	'MAGNOLIA, AR'
    elif value=='BHM'	: return	'BAR HARBOR, ME'
    elif value=='BHX'	: return	'BIRMINGHAM, AL'
    elif value=='CAK'	: return	'AKRON, OH'
    elif value=='FOK'	: return	'SUFFOLK COUNTY, NY'
    elif value== 'LND'	: return	'LANDER, WY'
    elif value=='MAR'	: return	'MARFA, TX'
    elif value== 'MLI'	: return	'MOLINE, IL'
    elif value=='RIV'	: return	'RIVERSIDE, CA'
    elif value=='RME'	: return	'ROME, NY'
    elif value=='VNY'	: return	'VAN NUYS, CA'
    elif value== 'YUM'	: return	'YUMA, AZ'
    elif value=='FRG'	: return	'Collapsed (FOK) 06/15'
    elif value=='HRL'	: return	'Collapsed (HLG) 06/15'
    elif value== 'ISP'	: return	'Collapsed (FOK) 06/15'
    elif value== 'JSJ'	: return	'Collapsed (SAJ) 06/15'
    elif value== 'BUS'	: return	'Collapsed (BUF) 06/15'
    elif value== 'IAG'	: return	'Collapsed (NIA) 06/15'
    elif value== 'PHN'	: return	'Collapsed (PHU) 06/15'
    elif value== 'STN'	: return	'Collapsed (STR) 06/15'
    elif value== 'VMB'	: return	'Collapsed (VNB) 06/15'
    elif value=='T01'	: return	'Collapsed (SEA) 06/15'
    elif value== 'PHF'	: return	'No PORT Code (PHF)'
    elif value== 'DRV'	: return	'No PORT Code (DRV)'
    elif value==  'FTB'	: return	'No PORT Code (FTB)'
    elif value== 'GAC'	: return	'No PORT Code (GAC)'
    elif value== 'NC8'	: return	'No PORT Code (NC8)'
    elif value== 'NYL'	: return	'No PORT Code (NYL)'
    elif value== 'OAI'	: return	'No PORT Code (OAI)'
    elif value== 'PCW'	: return	'No PORT Code (PCW)'
    elif value=='WA5'	: return	'No PORT Code (WAS)'
    elif value=='WTR'	: return	'No PORT Code (WTR)'
    elif value=='X96'	: return	'No PORT Code (X96)'
    elif value=='XNA'	: return	'No PORT Code (XNA)'
    elif value=='YGF'	: return	'No PORT Code (YGF)'
    elif value== '5T6'	: return	'No PORT Code (5T6)'
    elif value=='060'	: return	'No PORT Code (60)'
    elif value=='SP0'	: return	'No PORT Code (SP0)'
    elif value=='W55'	: return	'No PORT Code (W55)'
    elif value=='X44'	: return	'No PORT Code (X44)'
    elif value=='AUH'	: return	'No PORT Code (AUH)'
    elif value=='RYY'	: return	'No PORT Code (RYY)'
    elif value== 'SUS'	: return	'No PORT Code (SUS)'
    elif value=='74S'	: return	'No PORT Code (74S)'
    elif value=='ATW'	: return	'No PORT Code (ATW)'
    elif value=='CPX'	: return	'No PORT Code (CPX)'
    elif value=='MTH'	: return	'No PORT Code (MTH)'
    elif value== 'PFN'	: return	'No PORT Code (PFN)'
    elif value=='SCH'	: return	'No PORT Code (SCH)'
    elif value== 'ASI'	: return	'No PORT Code (ASI)'
    elif value==  'BKF'	: return	'No PORT Code (BKF)'
    elif value== 'BCM'	: return	'No PORT Code (BCM)'
    elif value== 'DEC'	: return	'No PORT Code (DEC)'
    elif value=='PLB'	: return	'No PORT Code (PLB)'
    elif value== 'JBQ'	: return	'No PORT Code (JBQ)'
    elif value== 'JIG'	: return	'No PORT Code (JIG)'
    elif value== 'OGS'	: return	'No PORT Code (OGS)'
    elif value=='TIW'	: return	'No PORT Code (TIW)'
    elif value=='OTS'	: return	'No PORT Code (OTS)'
    elif value=='AMT'	: return	'No PORT Code (AMT)'
    elif value== 'EGE'	: return	'No PORT Code (EGE)'
    elif value== 'GPI'	: return	'No PORT Code (GPI)'
    elif value== 'NGL'	: return	'No PORT Code (NGL)'
    elif value== 'OLM'	: return	'No PORT Code (OLM)'
    elif value== '.GA'	: return	'No PORT Code (.GA)'
    elif value== 'CLX'	: return	'No PORT Code (CLX)'
    elif value== 'CP '	: return	'No PORT Code (CP)'
    elif value== 'FSC'	: return	'No PORT Code (FSC)'
    elif value== 'NK' 	: return	'No PORT Code (NK)'
    elif value== 'ADU' 	: return	'No PORT Code (ADU)'
    elif value== 'AKT' 	: return	'No PORT Code (AKT)'
    elif value== 'LIT' 	: return	'No PORT Code (LIT)'
    elif value== 'A2A' 	: return	'No PORT Code (A2A)'
    elif value== 'OSN' 	: return	'No PORT Code (OSN)'
    elif value == 582: return  'MEXICO Air Sea, and Not Reported (I-94, no land arrivals)'
    elif value == 236: return 'AFGHANISTAN'
    elif value == 101: return  'ALBANIA'
    elif value == 316: return  'ALGERIA'
    elif value == 102: return   'ANDORRA'
    elif value == 324: return   'ANGOLA'
    elif value == 529: return  'ANGUILLA'
    elif value == 518: return   'ANTIGUA-BARBUDA'
    elif value == 687: return   'ARGENTINA '
    elif value == 151: return   'ARMENIA'
    elif value == 532: return   'ARUBA'
    elif value == 438: return   'AUSTRALIA'
    elif value == 103 : return   'AUSTRIA'
    elif value == 152 : return   'AZERBAIJAN'
    elif value == 512 : return   'BAHAMAS'
    elif value == 298 : return   'BAHRAIN'
    elif value == 274 : return   'BANGLADESH'
    elif value == 513 : return   'BARBADOS'
    elif value == 104 : return   'BELGIUM'
    elif value == 581 : return   'BELIZE'
    elif value ==386 : return   'BENIN'
    elif value ==509 : return   'BERMUDA'
    elif value ==153 : return   'BELARUS'
    elif value ==242 : return   'BHUTAN'
    elif value ==688 : return   'BOLIVIA'
    elif value ==717 : return   'BONAIRE, ST EUSTATIUS, SABA' 
    elif value ==164 : return   'BOSNIA-HERZEGOVINA'
    elif value ==336 : return   'BOTSWANA'
    elif value ==689 : return   'BRAZIL'
    elif value ==525 : return   'BRITISH VIRGIN ISLANDS'
    elif value ==217 : return   'BRUNEI'
    elif value ==105 : return   'BULGARIA'
    elif value ==393 : return   'BURKINA FASO'
    elif value ==243 : return   'BURMA'
    elif value ==375 : return   'BURUNDI'
    elif value ==310 : return   'CAMEROON'
    elif value ==326 : return   'CAPE VERDE'
    elif value ==526 : return   'CAYMAN ISLANDS'
    elif value ==383 : return   'CENTRAL AFRICAN REPUBLIC'
    elif value ==384 : return   'CHAD'
    elif value ==690 : return   'CHILE'
    elif value ==245 : return   'CHINA, PRC'
    elif value ==721 : return   'CURACAO' 
    elif value ==270 : return   'CHRISTMAS ISLAND'
    elif value ==271 : return   'COCOS ISLANDS'
    elif value ==691 : return   'COLOMBIA'
    elif value ==317 : return   'COMOROS'
    elif value ==385 : return   'CONGO'
    elif value ==467 : return   'COOK ISLANDS'
    elif value ==575 : return   'COSTA RICA'
    elif value ==165 : return   'CROATIA'
    elif value ==584 : return   'CUBA'
    elif value ==218 : return   'CYPRUS'
    elif value ==140 : return   'CZECH REPUBLIC'
    elif value ==723 : return   'FAROE ISLANDS (PART OF DENMARK)'  
    elif value ==108 : return   'DENMARK'
    elif value ==322 : return   'DJIBOUTI'
    elif value ==519 : return   'DOMINICA'
    elif value ==585 : return   'DOMINICAN REPUBLIC'
    elif value ==240 : return   'EAST TIMOR'
    elif value ==692 : return   'ECUADOR'
    elif value ==368 : return   'EGYPT'
    elif value ==576 : return   'EL SALVADOR'
    elif value ==399 : return   'EQUATORIAL GUINEA'
    elif value ==372 : return   'ERITREA'
    elif value ==109 : return   'ESTONIA'
    elif value ==369 : return   'ETHIOPIA'
    elif value ==604 : return   'FALKLAND ISLANDS'
    elif value ==413 : return   'FIJI'
    elif value ==110 : return   'FINLAND'
    elif value ==111 : return   'FRANCE'
    elif value ==601 : return   'FRENCH GUIANA'
    elif value ==411 : return   'FRENCH POLYNESIA'
    elif value ==387 : return   'GABON'
    elif value ==338 : return   'GAMBIA'
    elif value ==758 : return   'GAZA STRIP' 
    elif value ==154 : return   'GEORGIA'
    elif value ==112 : return   'GERMANY'
    elif value ==339 : return   'GHANA'
    elif value ==143 : return   'GIBRALTAR'
    elif value ==113 : return   'GREECE'
    elif value ==520 : return   'GRENADA'
    elif value ==507 : return   'GUADELOUPE'
    elif value ==577 : return   'GUATEMALA'
    elif value ==382 : return   'GUINEA'
    elif value ==327 : return   'GUINEA-BISSAU'
    elif value ==603 : return   'GUYANA'
    elif value ==586 : return   'HAITI'
    elif value ==726 : return   'HEARD AND MCDONALD IS.'
    elif value ==149 : return   'HOLY SEE/VATICAN'
    elif value ==528 : return   'HONDURAS'
    elif value ==206 : return   'HONG KONG'
    elif value ==114 : return   'HUNG: return ARY'
    elif value ==115 : return   'ICELAND'
    elif value ==213 : return   'INDIA'
    elif value ==759 : return   'INDIAN OCEAN AREAS (FRENCH)' 
    elif value ==729 : return   'INDIAN OCEAN TERRITORY' 
    elif value ==204 : return   'INDONESIA'
    elif value ==249 : return   'IRAN'
    elif value ==250 : return   'IRAQ'
    elif value ==116 : return   'IRELAND'
    elif value ==251 : return   'ISRAEL'
    elif value ==117 : return   'ITALY'
    elif value ==388 : return   'IVORY COAST'
    elif value ==514 : return   'JAMAICA'
    elif value ==209 : return   'JAPAN'
    elif value ==253 : return   'JORDAN'
    elif value ==201 : return   'KAMPUCHEA'
    elif value ==155 : return   'KAZAKHSTAN'
    elif value ==340 : return   'KENYA'
    elif value ==414 : return   'KIRIBATI'
    elif value ==732 : return   'KOSOVO' 
    elif value ==272 : return   'KUWAIT'
    elif value ==156 : return   'KYRGYZSTAN'
    elif value ==203 : return   'LAOS'
    elif value ==118 : return   'LATVIA'
    elif value ==255 : return   'LEBANON'
    elif value ==335 : return   'LESOTHO'
    elif value ==370 : return   'LIBERIA'
    elif value ==381 : return   'LIBYA'
    elif value ==119 : return   'LIECHTENSTEIN'
    elif value ==120 : return   'LITHUANIA'
    elif value ==121 : return   'LUXEMBOURG'
    elif value ==214 : return   'MACAU'
    elif value ==167 : return   'MACEDONIA'
    elif value ==320 : return   'MADAGASCAR'
    elif value ==345 : return   'MALAWI'
    elif value ==273 : return   'MALAYSIA'
    elif value ==220 : return   'MALDIVES'
    elif value ==392 : return   'MALI'
    elif value ==145 : return   'MALTA'
    elif value ==472 : return   'MARSHALL ISLANDS'
    elif value ==511 : return   'MARTINIQUE'
    elif value ==389 : return   'MAURITANIA'
    elif value ==342 : return   'MAURITIUS'
    elif value ==760 : return   'MAYOTTE (AFRICA - FRENCH)' 
    elif value ==473 : return   'MICRONESIA, FED. STATES OF'
    elif value ==157 : return   'MOLDOVA'
    elif value ==122 : return   'MONACO'
    elif value ==299 : return   'MONGOLIA'
    elif value ==735 : return   'MONTENEGRO' 
    elif value ==521 : return   'MONTSERRAT'
    elif value ==332 : return   'MOROCCO'
    elif value ==329 : return   'MOZAMBIQUE'
    elif value ==371 : return   'NAMIBIA'
    elif value ==440 : return   'NAURU'
    elif value ==257 : return   'NEPAL'
    elif value ==123 : return   'NETHERLANDS'
    elif value ==508 : return   'NETHERLANDS ANTILLES'
    elif value ==409 : return   'NEW CALEDONIA'
    elif value ==464 : return   'NEW ZEALAND'
    elif value ==579 : return   'NICARAGUA'
    elif value ==390 : return   'NIGER'
    elif value ==343 : return   'NIGERIA'
    elif value ==470 : return   'NIUE'
    elif value ==275 : return   'NORTH KOREA'
    elif value ==124 : return   'NORWAY'
    elif value ==256 : return   'OMAN'
    elif value ==258 : return   'PAKISTAN'
    elif value ==474 : return   'PALAU'
    elif value ==743 : return   'PALESTINE' 
    elif value ==504 : return   'PANAMA'
    elif value ==441 : return   'PAPUA NEW GUINEA'
    elif value ==693 : return   'PARAGUAY'
    elif value ==694 : return   'PERU'
    elif value ==260 : return   'PHILIPPINES'
    elif value ==416 : return   'PITCAIRN ISLANDS'
    elif value ==107 : return   'POLAND'
    elif value ==126 : return   'PORTUGAL'
    elif value ==297 : return   'QATAR'
    elif value ==748 : return   'REPUBLIC OF SOUTH SUDAN'
    elif value ==321 : return   'REUNION'
    elif value ==127 : return   'ROMANIA'
    elif value ==158 : return   'RUSSIA'
    elif value ==376 : return   'RWANDA'
    elif value ==128 : return   'SAN MARINO'
    elif value ==330 : return   'SAO TOME AND PRINCIPE'
    elif value ==261 : return   'SAUDI ARABIA'
    elif value ==391 : return   'SENEGAL'
    elif value ==142 : return   'SERBIA AND MONTENEGRO'
    elif value ==745 : return   'SERBIA' 
    elif value ==347 : return   'SEYCHELLES'
    elif value ==348 : return   'SIERRA LEONE'
    elif value ==207 : return   'SINGAPORE'
    elif value ==141 : return   'SLOVAKIA'
    elif value ==166 : return   'SLOVENIA'
    elif value ==412 : return   'SOLOMON ISLANDS'
    elif value ==397 : return   'SOMALIA'
    elif value ==373 : return   'SOUTH AFRICA'
    elif value ==276 : return   'SOUTH KOREA'
    elif value ==129 : return   'SPAIN'
    elif value ==244 : return   'SRI LANKA'
    elif value ==346 : return   'ST. HELENA'
    elif value ==522 : return   'ST. KITTS-NEVIS'
    elif value ==523 : return   'ST. LUCIA'
    elif value ==502 : return   'ST. PIERRE AND MIQUELON'
    elif value ==524 : return   'ST. VINCENT-GRENADINES'
    elif value ==716 : return   'SAINT BARTHELEMY' 
    elif value ==736 : return   'SAINT MARTIN' 
    elif value ==749 : return   'SAINT MAARTEN' 
    elif value ==350 : return   'SUDAN'
    elif value ==602 : return  'SURINAME'
    elif value ==351 : return   'SWAZILAND'
    elif value ==130 : return   'SWEDEN'
    elif value ==131 : return  'SWITZERLAND'
    elif value ==262 : return   'SYRIA'
    elif value ==268 : return   'TAIWAN'
    elif value ==159 : return   'TAJIKISTAN'
    elif value ==353 : return   'TANZANIA'
    elif value ==263 : return   'THAILAND'
    elif value ==304 : return   'TOGO'
    elif value ==417 : return   'TONGA'
    elif value ==516 : return   'TRINIDAD AND TOBAGO'
    elif value ==323 : return   'TUNISIA'
    elif value ==264 : return   'TURKEY'
    elif value ==161 : return   'TURKMENISTAN'
    elif value ==527 : return   'TURKS AND CAICOS ISLANDS'
    elif value ==420 : return   'TUVALU'
    elif value ==352 : return   'UGANDA'
    elif value ==162 : return   'UKRAINE'
    elif value ==296 : return   'UNITED ARAB EMIRATES'
    elif value ==135 : return   'UNITED KINGDOM'
    elif value ==695 : return   'URUGUAY'
    elif value ==163 : return   'UZBEKISTAN'
    elif value ==410 : return   'VANUATU'
    elif value ==696 : return   'VENEZUELA'
    elif value ==266 : return   'VIETNAM'
    elif value ==469 : return   'WALLIS AND FUTUNA ISLANDS'
    elif value ==757 : return   'WEST INDIES (FRENCH)' 
    elif value ==333 : return   'WESTERN SAHARA'
    elif value ==465 : return   'WESTERN SAMOA'
    elif value ==216 : return   'YEMEN'
    elif value ==139 : return   'YUGOSLAVIA'
    elif value ==301 : return   'ZAIRE'
    elif value ==344 : return   'ZAMBIA'
    elif value ==315 : return   'ZIMBABWE'
    elif value ==403 : return   'INVALID: AMERICAN SAMOA'
    elif value ==712 : return   'INVALID: ANTARCTICA' 
    elif value ==700 : return   'INVALID: BORN ON BOARD SHIP'
    elif value ==719 : return   'INVALID: BOUVET ISLAND (ANTARCTICA/NORWAY TERR.)'
    elif value ==574 : return   'INVALID: CANADA'
    elif value ==720 : return   'INVALID: CANTON AND ENDERBURY ISLS' 
    elif value ==106 : return  'INVALID: CZECHOSLOVAKIA'
    elif value ==739 : return   'INVALID: DRONNING MAUD LAND (ANTARCTICA-NORWAY)' 
    elif value ==394 : return   'INVALID: FRENCH SOUTHERN AND ANTARCTIC'
    elif value ==501 : return   'INVALID: GREENLAND'
    elif value ==404 : return   'INVALID: GUAM'
    elif value ==730 : return   'INVALID: INTERNATIONAL WATERS' 
    elif value ==731 : return   'INVALID: JOHNSON ISLAND' 
    elif value ==471 : return   'INVALID: MARIANA ISLANDS, NORTHERN'
    elif value ==737 : return   'INVALID: MIDWAY ISLANDS' 
    elif value ==753 : return   'INVALID: MINOR OUTLYING ISLANDS - USA'
    elif value ==740 : return   'INVALID: NEUTRAL ZONE (S. ARABIA/IRAQ)' 
    elif value ==710 : return   'INVALID: NON-QUOTA IMMIGRANT'
    elif value ==505 : return   'INVALID: PUERTO RICO'
    elif value == 0  : return   'INVALID: STATELESS'
    elif value ==705 : return   'INVALID: STATELESS'
    elif value ==583 : return   'INVALID: UNITED STATES'
    elif value ==407 : return   'INVALID: UNITED STATES'
    elif value ==999 : return   'INVALID: UNKNOWN'
    elif value ==239 : return   'INVALID: UNKNOWN COUNTRY'
    elif value ==134 : return   'INVALID: USSR'
    elif value ==506 : return   'INVALID: U.S. VIRGIN ISLANDS'
    elif value ==755 : return   'INVALID: WAKE ISLAND'  
    elif value ==311 : return   'Collapsed Tanzania (should not show)'
    elif value ==741 : return   'Collapsed Curacao (should not show)'
    elif value ==54 : return   'No Country Code (54)'
    elif value ==100 : return   'No Country Code (100)'
    elif value ==187 : return   'No Country Code (187)'
    elif value ==190 : return   'No Country Code (190)'
    elif value ==200 : return   'No Country Code (200)'
    elif value ==219 : return   'No Country Code (219)'
    elif value ==238 : return   'No Country Code (238)'
    elif value ==277 : return   'No Country Code (277)'
    elif value ==293 : return   'No Country Code (293)'
    elif value ==300 : return   'No Country Code (300)'
    elif value ==319 : return   'No Country Code (319)'
    elif value ==365 : return   'No Country Code (365)'
    elif value ==395 : return   'No Country Code (395)'
    elif value ==400 : return   'No Country Code (400)'
    elif value ==485 : return   'No Country Code (485)'
    elif value ==503 : return   'No Country Code (503)'
    elif value ==589 : return   'No Country Code (589)'
    elif value ==592 : return   'No Country Code (592)'
    elif value ==791 : return   'No Country Code (791)'
    elif value ==849 : return   'No Country Code (849)'
    elif value ==914 : return   'No Country Code (914)'
    elif value ==944 : return   'No Country Code (944)'
    elif value ==996 : return   'No Country Code (996)' 
    elif value == 'AL': return 'ALABAMA'
    elif value =='AK': return'ALASKA'
    elif value =='AZ': return'ARIZONA'
    elif value =='AR': return'ARKANSAS'
    elif value =='CA': return'CALIFORNIA'
    elif value =='CO': return'COLORADO'
    elif value =='CT': return'CONNECTICUT'
    elif value =='DE': return'DELAWARE'
    elif value =='DC': return'DIST. OF COLUMBIA'
    elif value =='FL': return'FLORIDA'
    elif value =='GA': return'GEORGIA'
    elif value =='GU': return'GUAM'
    elif value =='HI': return'HAWAII'
    elif value =='ID': return'IDAHO'
    elif value =='IL': return'ILLINOIS'
    elif value =='IN': return'INDIANA'
    elif value =='IA': return'IOWA'
    elif value =='KS': return'KANSAS'
    elif value =='KY': return'KENTUCKY'
    elif value =='LA': return'LOUISIANA'
    elif value =='ME': return'MAINE'
    elif value =='MD': return'MARYLAND'
    elif value =='MA': return'MASSACHUSETTS'
    elif value =='MI': return'MICHIGAN'
    elif value =='MN': return'MINNESOTA'
    elif value =='MS': return'MISSISSIPPI'
    elif value =='MO': return'MISSOURI'
    elif value =='MT': return'MONTANA'
    elif value =='NC': return'N. CAROLINA'
    elif value =='ND': return'N. DAKOTA'
    elif value =='NE': return'NEBRASKA'
    elif value =='NV': return'NEVADA'
    elif value =='NH': return'NEW HAMPSHIRE'
    elif value =='NJ': return'NEW JERSEY'
    elif value =='NM': return'NEW MEXICO'
    elif value =='NY': return'NEW YORK'
    elif value =='OH': return'OHIO'
    elif value =='OK': return'OKLAHOMA'
    elif value =='OR': return'OREGON'
    elif value =='PA': return'PENNSYLVANIA'
    elif value =='PR': return'PUERTO RICO'
    elif value =='RI': return'RHODE ISLAND'
    elif value =='SC': return'S. CAROLINA'
    elif value =='SD': return'S. DAKOTA'
    elif value =='TN': return'TENNESSEE'
    elif value =='TX': return'TEXAS'
    elif value =='UT': return'UTAH'
    elif value =='VT': return'VERMONT'
    elif value =='VI': return'VIRGIN ISLANDS'
    elif value =='VA': return'VIRGINIA'
    elif value =='WV': return'W. VIRGINIA'
    elif value =='WA': return'WASHINGTON'
    elif value =='WI': return'WISCONSON'
    elif value =='WY': return'WYOMING' 
    elif value =='99': return'All Other Codes'
    elif value == 1: return 'Air'
    elif value == 2: return 'Sea'
    elif value == 3: return 'Land'
    elif value == 9: return 'Not Reported'
    elif value == 1: return 'Business'
    elif value == 2: return 'Pleasure'
    elif value == 3: return 'Student'
    else: return 'Other'

In [6]:
udfvalueToCategory = udf(valuetocategory, StringType())

In [7]:
visitor_mod_of_arrival_df = df_spark.withColumn('i94mode',df_spark.i94mode.cast("int"))\
                                    .withColumn('category',udfvalueToCategory('i94mode'))\
                                    .withColumn('airline',df_spark.airline)\
                                    .withColumn('fltno',df_spark.fltno)

visitor_mod_of_arrival_df = visitor_mod_of_arrival_df.withColumn('i94mode',visitor_mod_of_arrival_df.i94mode)\
                                                     .withColumn('category',visitor_mod_of_arrival_df.category)\
                                                     .withColumn('airline',visitor_mod_of_arrival_df.airline)\
                                                     .withColumn('fltno',visitor_mod_of_arrival_df.fltno)

visitor_mod_arrival = visitor_mod_of_arrival_df[['i94mode','category','airline','fltno']].dropDuplicates()

In [8]:
#visitor_mod_arrival.createOrReplaceTempView("Visitor_Arrival_Modes")

In [9]:
#spark.sql("select count(*) from Visitor_Arrival_Modes").show()

In [10]:
#visitor_mod_arrival = visitor_mod_arrival.write.partitionBy("i94mode","airline","fltno").parquet("Immigration-Warehouse/Visitor_Arrival_Modes/")

In [11]:
#visitor_mod_arrival.orderBy("i94_mode_of_arrival_id","i94mode").show(500)

In [12]:
#df_spark = df_spark[df_spark.i94mode.isin([1,2,3])].count()

In [23]:
#udfcountryvalueToCategory = udf(countryvaluetocategory, StringType())
                             # .withColumn('age',df_spark.i94bir.cast("int"))\
                              #.withColumn('gender',df_spark.gender)\

In [14]:
visitor_personal_df = df_spark.withColumn('i94cit',df_spark.i94cit.cast("int"))\
                              .withColumn('travel_city',udfvalueToCategory('i94cit'))\
                              .withColumn('i94res',df_spark.i94res.cast("int"))\
                              .withColumn('residence',udfvalueToCategory('i94res'))\
                              .withColumn('visitor_age',df_spark.i94bir.cast("int"))\
                              .withColumn('visitor_gender',df_spark.gender)\

visitor_personal_df = visitor_personal_df.withColumn('i94cit',visitor_personal_df.i94cit)\
                                         .withColumn('travel_city',visitor_personal_df.travel_city)\
                                         .withColumn('i94res',visitor_personal_df.i94res)\
                                         .withColumn('residence',visitor_personal_df.residence)\
                                         .withColumn('visitor_age',visitor_personal_df.visitor_age)\
                                         .withColumn('visitor_gender',visitor_personal_df.visitor_gender)\

visitor_table = visitor_personal_df[['i94cit','travel_city','i94res','residence','visitor_age','visitor_gender']].dropDuplicates()

In [15]:
visitor_table.count()

104509

In [29]:
#visitor_table.createOrReplaceTempView("visitor_details")

In [26]:
#spark.sql("select count(*) from visitor_details").show()

In [1]:
#visitor_table = visitor_table.write.partitionBy("i94cit","i94res").parquet("Immigration-Warehouse/Visitor_Details/")

In [2]:
#visitor_table.orderBy("visitor_id","i94cit","i94res").show(500)

In [27]:
#df_spark = df_spark[df_spark['admnum'] == '0.0']

In [28]:
#visitor_personal_df = visitor_personal_df[visitor_personal_df['i94cit'] == 585]

In [16]:
visitor_i94_df = df_spark.withColumn('i94_addr',df_spark.i94addr)\
                         .withColumn('i94_addr_description',udfvalueToCategory('i94addr'))\
                         .withColumn('i94_port',df_spark.i94port)\
                         .withColumn('i94_port_description',udfvalueToCategory('i94port'))\

visitor_i94_df = visitor_i94_df.withColumn('i94_addr',visitor_i94_df.i94addr)\
                               .withColumn('i94_addr_description',visitor_i94_df.i94_addr_description)\
                               .withColumn('i94_port',visitor_i94_df.i94port)\
                               .withColumn('i94_port_description',visitor_i94_df.i94_port_description)\

visitor_i94_codes = visitor_i94_df['i94_addr','i94_addr_description','i94_port','i94_port_description'].dropDuplicates()

visitor_i94_codes = visitor_i94_codes.na.fill({'i94_addr':'OT'})

In [17]:
#visitor_i94_codes.count()

In [18]:
#visitor_i94_codes.createOrReplaceTempView("visitor_i94_codes")

In [19]:
#spark.sql("select count(*) from visitor_i94_codes").show()

In [20]:
#visitor_i94_codes = visitor_i94_codes.write.partitionBy("i94_addr","i94_port").parquet("Immigration-Warehouse/Visitor_I94_Codes/")

In [21]:
#visitor_i94_codes.orderBy("i94_code_id","i94_addr","i94_port").show(500)

In [22]:
#visitor_i94_df = visitor_i94_df[visitor_i94_df['i94_code_id'] == 1]

In [23]:
visitor_visa_df = df_spark.withColumn('i94_visa',df_spark.i94visa.cast("int"))\
                              .withColumn('i94_visa_description',udfvalueToCategory('i94visa'))\
                              .withColumn('visa_type',df_spark.visatype)\

visitor_visa = visitor_visa_df['i94_visa','i94_visa_description','visa_type'].dropDuplicates()

In [24]:
#visitor_visa.createOrReplaceTempView("visitor_visa")

In [25]:
#spark.sql("select count(*) from visitor_visa").show()

In [56]:
#visitor_visa = visitor_visa.write.partitionBy("i94_visa","visa_type").parquet("Immigration-Warehouse/Visitor_Visa_Details/")

In [27]:
df_spark = df_spark.fillna({'depdate':0})

time = spark.udf.register("get_ts", lambda x: (datetime.timedelta(days=x) + datetime.datetime(1960,1,1)).strftime('%Y-%m-%d'))

visitor_time_df = df_spark.withColumn('i94_admission_num',df_spark.admnum)\
                          .withColumn('i94_year',df_spark.i94yr.cast("int"))\
                          .withColumn('i94_month',df_spark.i94mon.cast("int"))\
                          .withColumn('i94_arrival_date',time(df_spark.arrdate.cast("int")))\
                          .withColumn('i94_departure_date',time(df_spark.depdate.cast("int")))\
                          .withColumn('i94_date_added_on_file',to_date(unix_timestamp(df_spark.dtadfile, 'yyyyMMdd').cast("timestamp")))\
                          .withColumn('i94_date_admitted_until',to_date(unix_timestamp(df_spark.dtaddto, 'MMddyyyy').cast("timestamp")))\

visitor_time = visitor_time_df['i94_admission_num','i94_year','i94_month','i94_arrival_date','i94_departure_date','i94_date_added_on_file','i94_date_admitted_until'].dropDuplicates()

visitor_time = visitor_time.na.replace(['1960-01-01'], ['null'], 'i94_departure_date')

In [28]:
#visitor_time.createOrReplaceTempView("visitor_times")

In [29]:
#spark.sql("select count(*) from visitor_times").show()

In [30]:
#visitor_time = visitor_time.write.partitionby("i94_year","i94_month").parquet("Immigration-Warehouse/Visitor_Times/")

In [31]:
#df_spark = df_spark[df_spark['admnum'] == '9.501354793E10']
#df_spark = df_spark[df_spark['admnum'] == '6.9918985E7']

In [32]:
df_spark = df_spark.fillna({'i94addr':'OT'})

In [33]:
df_spark.count()

3096313

In [34]:
visitor_mod_arrival.count()

12020

In [35]:
visitor_table.count()

104509

In [36]:
visitor_i94_codes.count()

6638

In [37]:
visitor_visa.count()

17

In [38]:
visitor_time.count()

3096241

In [41]:
i94_df = df_spark.join(broadcast(visitor_mod_arrival),(df_spark.i94mode.cast("int") == visitor_mod_arrival.i94mode)&(df_spark.airline == visitor_mod_arrival.airline)&(df_spark.fltno == visitor_mod_arrival.fltno),"inner")\
                 .join(broadcast(visitor_table),(df_spark.i94cit.cast("int") == visitor_table.i94cit)&(df_spark.i94res == visitor_table.i94res)&(df_spark.i94bir.cast("int") == visitor_table.visitor_age)&(df_spark.gender == visitor_table.visitor_gender),"inner")\
                 .join(broadcast(visitor_i94_codes),(df_spark.i94port == visitor_i94_codes.i94_port)&(df_spark.i94addr == visitor_i94_codes.i94_addr),"inner")\
                 .join(broadcast(visitor_visa),(df_spark.i94visa.cast("int") == visitor_visa.i94_visa)&(df_spark.visatype == visitor_visa.visa_type),"inner")\
                 .join(broadcast(visitor_time),(df_spark.admnum == visitor_time.i94_admission_num)&(df_spark.i94yr.cast("int") == visitor_time.i94_year)&(df_spark.i94mon.cast("int") == visitor_time.i94_month),"inner")\

In [42]:
i94_df.count()

2623556

In [44]:
    i94_df = i94_df.withColumn('i94_admission_num',df_spark.admnum)\
                   .withColumn('i94_arrival_date',i94_df.i94_arrival_date)\
                   .withColumn('visa_type',i94_df.visa_type)\
                   .withColumn('i94_date_admitted_until',i94_df.i94_date_admitted_until)\
                   .withColumn('i94_port_description',i94_df.i94_port_description)\
                   .withColumn('i94_addr_description',i94_df.i94_addr_description)\
                   .withColumn('travel_city',i94_df.travel_city)\
                   .withColumn('residence',i94_df.residence)\
                   .withColumn('category',i94_df.category)\
                   .withColumn('visitor_age',i94_df.visitor_age)\
                   .withColumn('birth_year',df_spark.biryear.cast("int"))\
                   .withColumn('visitor_gender',i94_df.visitor_gender)\
                   .withColumn('insurance_number',df_spark.insnum)

In [45]:
    i94_df = i94_df.withColumn('i94_admission_num',i94_df.i94_admission_num)\
                   .withColumn('i94_arrival_date',i94_df.i94_arrival_date)\
                   .withColumn('visa_type',i94_df.visa_type)\
                   .withColumn('i94_date_admitted_until',i94_df.i94_date_admitted_until)\
                   .withColumn('i94_port_description',i94_df.i94_port_description)\
                   .withColumn('i94_addr_description',i94_df.i94_addr_description)\
                   .withColumn('travel_city',i94_df.travel_city)\
                   .withColumn('residence',i94_df.residence)\
                   .withColumn('category',i94_df.category)\
                   .withColumn('visitor_age',i94_df.visitor_age)\
                   .withColumn('birth_year',i94_df.birth_year)\
                   .withColumn('visitor_gender',i94_df.visitor_gender)\
                   .withColumn('insurance_number',i94_df.insurance_number)

In [46]:
#i94_df.count()

In [47]:
i94_table = i94_df[['i94_admission_num','i94_arrival_date','visa_type','i94_date_admitted_until',
                        'i94_port_description','i94_addr_description','travel_city','residence',
                        'category','visitor_age','birth_year','visitor_gender','insurance_number']].dropDuplicates()

In [48]:
i94_table.count()

2618419

In [77]:
#i94_df.createOrReplaceTempView('i94_immigration_data')

In [58]:
#i94_table.orderBy("i94_admission_num","i94_arrival_date").show(5)
#&(df_spark.admnum == visitor_time.i94_admission_number)

In [49]:
i94_table.createOrReplaceTempView('i94_immigration_data')

In [50]:
spark.sql('select count(*) from i94_immigration_data').show()

+--------+
|count(1)|
+--------+
| 2618419|
+--------+



In [51]:
spark.sql('select count(i94_admission_num) total_admissions,i94_port_description from i94_immigration_data group by i94_port_description order by total_admissions desc').show()

+----------------+--------------------+
|total_admissions|i94_port_description|
+----------------+--------------------+
|          399416|NEW YORK, NY     ...|
|          300918|MIAMI, FL        ...|
|          264674|LOS ANGELES, CA  ...|
|          141297|SAN FRANCISCO, CA...|
|          134659|NEWARK/TETERBORO,...|
|          123337|ORLANDO, FL      ...|
|          113032|HONOLULU, HI     ...|
|          109040|CHICAGO, IL      ...|
|           93586|HOUSTON, TX      ...|
|           84160|FORT LAUDERDALE, ...|
|           81476|AGANA, GU        ...|
|           80822|LAS VEGAS, NV    ...|
|           65558|ATLANTA, GA      ...|
|           64491|WASHINGTON DC    ...|
|           54499|DALLAS, TX       ...|
|           44326|BOSTON, MA       ...|
|           40239|SEATTLE, WA      ...|
|           34178|PHOENIX, AZ      ...|
|           26554|DETROIT, MI      ...|
|           24035|PHILADELPHIA, PA ...|
+----------------+--------------------+
only showing top 20 rows



In [54]:
spark.sql('select count(i94_admission_num) total_admissions,travel_city,residence from i94_immigration_data where visitor_gender = "M" and visitor_age < 30 group by travel_city,residence order by total_admissions desc').show()

+----------------+--------------------+--------------------+
|total_admissions|         travel_city|           residence|
+----------------+--------------------+--------------------+
|           31715|      UNITED KINGDOM|      UNITED KINGDOM|
|           23683|              FRANCE|              FRANCE|
|           18714|MEXICO Air Sea, a...|MEXICO Air Sea, a...|
|           17085|          CHINA, PRC|          CHINA, PRC|
|           16369|               JAPAN|               JAPAN|
|           15549|               Other|         SOUTH KOREA|
|           12454|              BRAZIL|              BRAZIL|
|           11149|               Other|             GERMANY|
|           10701|           AUSTRALIA|           AUSTRALIA|
|            8958|               INDIA|               INDIA|
|            7949|         NETHERLANDS|         NETHERLANDS|
|            7622|          ARGENTINA |          ARGENTINA |
|            6591|             ECUADOR|             ECUADOR|
|            6560|      

In [55]:
spark.sql('select count(i94_admission_num) total_admissions,i94_addr_description from i94_immigration_data where i94_date_admitted_until is null group by i94_addr_description order by total_admissions desc').show()

+----------------+--------------------+
|total_admissions|i94_addr_description|
+----------------+--------------------+
|            9865|          CALIFORNIA|
|            6831|            NEW YORK|
|            3858|             FLORIDA|
|            2930|       MASSACHUSETTS|
|            1822|               TEXAS|
|            1583|            ILLINOIS|
|            1518|          WASHINGTON|
|            1131|               Other|
|             985|        PENNSYLVANIA|
|             884|          NEW JERSEY|
|             725|            MICHIGAN|
|             708|             GEORGIA|
|             700|            VIRGINIA|
|             632|   DIST. OF COLUMBIA|
|             513|            MARYLAND|
|             478|                OHIO|
|             458|                UTAH|
|             445|        RHODE ISLAND|
|             419|         CONNECTICUT|
|             372|             INDIANA|
+----------------+--------------------+
only showing top 20 rows



In [ ]:
#df_spark.select(df_spark['i94cit'] = 254.0).show()

In [ ]:
#write to parquet
#df_spark.write.parquet("sas_data")
#df_spark=spark.read.parquet("sas_data")